# Linear regression with Professor Mittens

In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import altair as alt

## Coefficient estimation in ordinary linear regression **(10 + 10 minutes)**

In the lectures you have seen how to compute the least-squares estimate for ordinary linear regression. In practice, you'll probably want to use a library which implements this functionality rather than coding it yourself. A very popular way to describe regression models is with "formulae" as popularised by R. The [R documentation on formulae](https://cran.r-project.org/doc/manuals/R-intro.html#Formulae-for-statistical-models) is a good place to learn how to use these properly.

### Example

Suppose you have a data frame with columns called `Y` and `X`. The formular for a model $Y = a + bX + \epsilon$ is given by `Y ~ 1 + X`.

### Exercise

Consider the following data set. Fit a linear model for `Y` with the following sets of covariates: none, just `X`, just `Z`, both `X` and `Z`, and both `X`, `Z` and their interaction terms.

In [ ]:
# Here we just construct the data set that is needed for this example.

In [3]:
## YOUR CODE HERE!!!

## The assumptions of ordinary linear regression

- *20 minutes + 15 minutes, 2 questions*
- Review the basic assumptions of linear regression and why these are important,
  noting that only some of them can be observed from the data.
  + linearity
  + constant variance
  + normality
  + independence of errors
  + no error in the measurement of the covariate
  + the covariate is not invariant
- Get them to visualise the data and model fit 

# SHORT BREAK

Please take a 5 minute breather

## Interrogating the results of ordinary linear regression

- *15 minutes + 10 minutes, 2 questions*
- Get them to write a function to compute model summary statistics which are
  useful for assessing model fit.
  + correlation coefficient
  + residual standard error
  + coefficient of determination
- Get them to understand the different between a significant effect and a
  substantial effect.

## Checking data for violation of assumptions

- *25 minutes + 10 minutes, 6 questions*
- Get them to work though data sets working out in what ways they violate the
  assumptions of linear regression and to estimate the parameters for the couple
  of examples where they follow the assumptions.

## Linear regression with multiple covariates

- Diagnose an example where there is very strong correlation between two of the
  covariates.